In [4]:
# !pip3 install --upgrade pip
# !pip3 install ultralytics opencv-python



In [3]:
# download the dataset: https://www.kaggle.com/datasets/andrewmvd/car-plate-detection?resource=
# rename the folder: "dataset"

In [ ]:
import os
import shutil
import random

In [ ]:
random.seed(42)

# Input directories
image_dir = 'dataset/images'
label_dir = 'dataset/annotations'

# Output base directory
out_base = 'dataset_split'

# Collect all image base names
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
base_names = [os.path.splitext(f)[0] for f in image_files]

# Shuffle and split
random.shuffle(base_names)
total = len(base_names)
train_end = int(0.7 * total)
val_end = int(0.9 * total)

splits = {
    'train': base_names[:train_end],
    'val': base_names[train_end:val_end],
    'test': base_names[val_end:]
}

# Create target folders
for split in ['train', 'val', 'test']:
    os.makedirs(f'{out_base}/images/{split}', exist_ok=True)
    os.makedirs(f'{out_base}/labels/{split}', exist_ok=True)

# Move files
for split, names in splits.items():
    for name in names:
        img_src = os.path.join(image_dir, f'{name}.png')
        lbl_src = os.path.join(label_dir, f'{name}.xml')
        
        img_dst = os.path.join(out_base, 'images', split, f'{name}.png')
        lbl_dst = os.path.join(out_base, 'labels', split, f'{name}.xml')
        
        if os.path.exists(img_src):
            shutil.copy(img_src, img_dst)
        if os.path.exists(lbl_src):
            shutil.copy(lbl_src, lbl_dst)

print("Files split and grouped")


Files split and grouped
